In [ ]:
import itertools
import functools
import collections
import operator
import sys
import re
import copy
import numpy as np
import math

In [ ]:
from helpers.functions import *

Configuration

In [ ]:
DIR = "data/2019/"
load_day = functools.partial(load, DIR)

# Problems

## Day 1

http://adventofcode.com/2019/day/1

In [ ]:
content = mapl(int, load_day(1))

__Part 1__

In [ ]:
def get_fuel(mass):
    return max(0, math.floor(mass/3) - 2)

In [ ]:
sum(map(get_fuel, content))

__Part 2__

In [ ]:
def get_over_all_fuel(mass):
    if mass <= 0:
        return 0
    
    fuel = get_fuel(mass)
    return fuel + get_over_all_fuel(fuel)

In [ ]:
sum(map(get_over_all_fuel, content))

## Day 2

http://adventofcode.com/2019/day/2

In [ ]:
content = mapl(int, load_day(2)[0].split(","))

__Part 1__

In [ ]:
def process(program):
    i = 0

    while True:
        if program[i] == 99:
            break

        opcode, idx1, idx2, idx3 = program[i:i+4]

        if opcode == 1:
            program[idx3] = program[idx1] + program[idx2]
        elif opcode == 2:
            program[idx3] = program[idx1] * program[idx2]
        else:
            raise ValueError("Unkown opcode")

        i += 4
        
    return program[0]

In [ ]:
program = content.copy()
program[1] = 12
program[2] = 2
process(program)

__Part 2__

In [ ]:
for i in range(100):
    for j in range(100):
        program = content.copy()
        program[1] = i
        program[2] = j
        if process(program) == 19690720:
            print(100*i+j)
            raise

## Day 3

http://adventofcode.com/2019/day/3

In [ ]:
content = load_day(3)

In [ ]:
path1 = content[0].split(",")
path2 = content[1].split(",")

In [ ]:
directions = {
    "R": (1,0), 
    "L": (-1,0),
    "U": (0,1),
    "D": (0,-1),
}

def get_positions(path):
    positions = list()
    current_position = (0,0)
    for step in path:
        direction, number = step[0], int(step[1:])
        for i in range(number):
            current_position = tuple(add(current_position, directions[direction]))
            positions.append(current_position)
            
    return positions

In [ ]:
positions_path1 = get_positions(path1)
positions_path2 = get_positions(path2)

In [ ]:
commun = set(positions_path1).intersection(set(positions_path2))

__Part 1__

In [ ]:
min([abs(x)+abs(y) for x, y in commun])

__Part 2__

In [ ]:
steps = []
for c in commun:
    idx1 = positions_path1.index(c) + 1
    idx2 = positions_path2.index(c) + 1
    steps.append(idx1 + idx2)

In [ ]:
min(steps)

## Day 4

http://adventofcode.com/2019/day/4

In [ ]:
low = 356261
high = 846303

In [ ]:
def never_decrease(numbers):
    return all(x <= y for x,y in zip(numbers, numbers[1:]))

def contains_a_pair(numbers):
    return any(x == y for x,y in zip(numbers, numbers[1:]))

In [ ]:
pwds = []
for pwd in range(low, high+1):
    numbers = mapl(int, str(pwd))
    if never_decrease(numbers) and contains_a_pair(numbers):
        pwds.append(numbers)

__Part 1__

In [ ]:
len(pwds)

__Part 2__

In [ ]:
sum(1 for pwd in pwds if 2 in collections.Counter(pwd).values())

## Day 5

http://adventofcode.com/2019/day/5

In [ ]:
instructions = mapl(int, load_day(5)[0].split(","))

In [ ]:
def intcode_computer(instructions, inputs, ptr=0):
    output = None

    def params(ptr, mode):
        return instructions[instructions[ptr]] if mode == 0 else instructions[ptr]

    while True:
        opcode = instructions[ptr] % 100
        param1_mode = (instructions[ptr] % 1000) // 100
        param2_mode = (instructions[ptr] % 10000) // 1000

        if opcode == 1:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            c = instructions[ptr+3]
            instructions[c] = a + b
            ptr += 4

        elif opcode == 2:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            c = instructions[ptr+3]
            instructions[c] = a * b
            ptr += 4

        elif opcode == 3:
            instructions[instructions[ptr+1]] = inputs.pop(0)
            ptr += 2

        elif opcode == 4:
            output = params(ptr+1, param1_mode)
            ptr += 2
            break
        elif opcode == 5:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            ptr = b if a != 0 else ptr+3

        elif opcode == 6:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            ptr = b if a == 0 else ptr+3

        elif opcode == 7:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            c = instructions[ptr+3]
            instructions[c] = int(a < b)
            ptr += 4

        elif opcode == 8:
            a = params(ptr+1, param1_mode)
            b = params(ptr+2, param2_mode)
            c = instructions[ptr+3]
            instructions[c] = int(a == b)
            ptr += 4

        elif opcode == 99:
            break

        else:
            print(opcode)
            raise
            
    return output, ptr

__Part 1__

In [ ]:
ptr = 0
instr = instructions.copy()
diag = None
while True:
    new_diag, ptr = intcode_computer(instr, [1], ptr)
    if new_diag is None:
        print(diag)
        break
    diag = new_diag

__Part 2__

In [ ]:
diag, ptr = intcode_computer(instructions.copy(), [5])
diag

## Day 6

http://adventofcode.com/2019/day/6

In [ ]:
content = mapl(lambda x: x.split(")"), load_day(6))

In [ ]:
parents = dict()
childrens = collections.defaultdict(set)

for u,v in content:
    parents[v] = u
    childrens[u].add(v)

__Part 1__

In [ ]:
total = 0

for obj in parents.keys():
    while obj != "COM":
        total += 1
        obj = parents[obj]
        
total

__Part 2__

In [ ]:
start = "YOU"
end = "SAN"

explored = set()

def explore(source, root, transfers):
    explored.add(root)
    
    if root == "SAN":
        return transfers
    
    nodes = childrens[root]
    if root in parents: 
        nodes.add(parents[root])
    
    for node in nodes:
        if node not in explored:
            out = explore(root, node, transfers+1)
            if out is not None:
                return out

In [ ]:
explore(start, parents[start], 0) - 1

## Day 7

http://adventofcode.com/2019/day/7

In [ ]:
instructions = mapl(int, load_day(7)[0].split(","))

__Part 1__

In [ ]:
max_amp = 0
for phases in itertools.permutations(range(5)):
    amp = 0
    for p in phases:
        amp, ptr = intcode_computer(instructions, [p, amp])
    
    max_amp = max(max_amp, amp)

In [ ]:
max_amp

__Part 2__

In [ ]:
max_amp = 0
for phases in itertools.permutations(range(5,10)):
    amp_instructions = [instructions.copy() for _ in range(5)]
    amp_inputs = [[p] for p in phases]
    amp_ptr = [0, 0, 0, 0, 0]
    
    amp = 0

    while amp is not None:
        for i in range(5):
            amp_inputs[i].append(amp)
            amp, amp_ptr[i] = intcode_computer(amp_instructions[i], amp_inputs[i], ptr=amp_ptr[i])
        
        last_amp = last_amp if amp is None else amp
        
    max_amp = max(max_amp, last_amp)

In [ ]:
max_amp

## Day 8

http://adventofcode.com/2019/day/8

In [ ]:
content = load_day(8)[0]

In [ ]:
width = 25
height = 6
layers_count = (len(content) // width) // height

In [ ]:
pixels = mapl(int, content)
layers = np.zeros((layers_count, height, width))

for l in range(layers_count):   
    for y in range(height):
        for x in range(width):
            layers[l, y, x] = int(pixels.pop(0))

__Part 1__

In [ ]:
min_zeros = sys.maxsize
output = None
for l in range(layers_count):
    counter = collections.Counter(layers[l].flatten())
    if counter[0] < min_zeros:
        output = counter[1] * counter[2]
        min_zeros = counter[0]

In [ ]:
output

__Part 2__

In [ ]:
for y in range(height):
    for x in range(width):
        val = next(e for e in np.array(layers)[:, y, x] if e != 2)
        
        if val == 2:
            print("?", end="")
        elif val == 1:
            print("#", end="")
        elif val == 0:
            print(" ", end="")
    print()

## Day 9

http://adventofcode.com/2019/day/9

In [ ]:
content = load_day(9)

__Part 1__

__Part 2__

## Day 10

http://adventofcode.com/2019/day/10

In [ ]:
asteroids_belt = np.array(mapl(list, load_day(10)))

In [ ]:
asteroids = set()
for x in range(asteroids_belt.shape[0]):
    for y in range(asteroids_belt.shape[1]):
        if asteroids_belt[x, y] == "#":
            asteroids.add((y, x))

In [ ]:
stations2neighbors = collections.defaultdict(set)

for sy, sx in asteroids:
    for ay, ax in asteroids:
        if (sy, sx) == (ay, ax):
            continue
            
        y, x = (ay - sy, ax - sx)
        gcd = abs(math.gcd(x, y))
        stations2neighbors[(sy, sx)].add((y // gcd, x // gcd))

__Part 1__

In [ ]:
max_value, station = max((len(x), pos) for pos, x in stations2neighbors.items())

In [ ]:
max_value, station

__Part 2__

In [ ]:
to_zap = sorted(((math.atan2(dy, dx), (dy, dx)) for dy, dx in stations2neighbors[station]), reverse=True)

_, (dy, dx) = to_zap[200-1]

y, x = station[0] + dy, station[1] + dx
while (x, y) not in asteroids:
    x, y = x + dx, y + dy

In [ ]:
y*100 + x

## Day 11

http://adventofcode.com/2019/day/11

In [ ]:
content = load_day(11)

__Part 1__

__Part 2__

## Day 12

http://adventofcode.com/2019/day/12

In [ ]:
content = load_day(12)

__Part 1__

__Part 2__

## Day 13

http://adventofcode.com/2019/day/13

In [ ]:
content = load_day(13)

__Part 1__

__Part 2__

## Day 14

http://adventofcode.com/2019/day/14

In [ ]:
content = load_day(14)

__Part 1__

__Part 2__

## Day 15

http://adventofcode.com/2019/day/15

In [ ]:
content = load_day(15)

__Part 1__

__Part 2__

## Day 16

http://adventofcode.com/2019/day/16

In [ ]:
content = load_day(16)

__Part 1__

__Part 2__

## Day 17

http://adventofcode.com/2019/day/17

In [ ]:
content = load_day(17)

__Part 1__

__Part 2__

## Day 18

http://adventofcode.com/2019/day/18

In [ ]:
content = load_day(18)

__Part 1__

__Part 2__

## Day 19

http://adventofcode.com/2019/day/19

In [ ]:
content = load_day(19)

__Part 1__

__Part 2__

## Day 20

http://adventofcode.com/2019/day/20

In [ ]:
content = load_day(20)

__Part 1__

__Part 2__

## Day 21

http://adventofcode.com/2019/day/21

In [ ]:
content = load_day(21)

__Part 1__

__Part 2__

## Day 22

http://adventofcode.com/2019/day/22

In [ ]:
content = load_day(22)

__Part 1__

__Part 2__

## Day 23

http://adventofcode.com/2019/day/23

In [ ]:
content = load_day(23)

__Part 1__

__Part 2__

## Day 24

http://adventofcode.com/2019/day/24

In [ ]:
content = load_day(24)

__Part 1__

__Part 2__

## Day 25

http://adventofcode.com/2019/day/25

In [ ]:
content = load_day(25)

__Part 1__

__Part 2__